In [1]:
import pytorch_lightning as pl
import numpy as np
import torchaudio
import torch
import torch.optim
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd

from torch.utils.data import DataLoader
from torch import nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoderLayer
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
from utils import AudioDataset, WaveNet, AudioMELSpectogramDataset

e:\miniconda3\envs\AI\lib\site-packages\torch\distributed\_shard\partial_tensor.py:40: UserWarning: Please use DTensor instead and we are deprecating ShardedTensor.
  warnings.warn(DEPRECATE_MSG)
e:\miniconda3\envs\AI\lib\site-packages\torch\distributed\_shard\replicated_tensor.py:20: UserWarning: Please use DTensor instead and we are deprecating ShardedTensor.
  warnings.warn(DEPRECATE_MSG)


In [2]:
class StyleAutoEncoder(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, dilation, n_head=8, dropout=0.2):
        super(StyleAutoEncoder, self).__init__()
        self.sharedEncoder = WaveNet(in_channels, out_channels//2, kernel_size, dilation, dropout)
        self.styleEncoder = WaveNet(out_channels//2, out_channels, kernel_size, dilation, dropout)
        self.speechEncoder = WaveNet(out_channels//2, out_channels, kernel_size, dilation, dropout)
        self.style_encoder_attention = TransformerEncoderLayer(out_channels, nhead=2)
        self.speech_encoder_attention = TransformerEncoderLayer(out_channels, nhead=2)
        self.decoder_attention = TransformerEncoderLayer(out_channels*2, nhead=2)
        self.decoder = WaveNet(out_channels*2, in_channels, kernel_size, dilation, dropout)
        # self.wav2vec = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
        # self.tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")

    def forward(self, speech, style):

        style = self.sharedEncoder(style)
        speech = self.sharedEncoder(speech) 

        style = self.styleEncoder(style)
        speech = self.speechEncoder(speech)
        
        style = style.permute(2, 0, 1)  # Reordenamos las dimensiones para que sea compatible con la capa de atención
        style = self.style_encoder_attention(style)
        style = style.permute(1, 2, 0)  # Restauramos el orden de las dimensiones
        
        speech = speech.permute(2, 0, 1)  # Reordenamos las dimensiones para que sea compatible con la capa de atención
        speech = self.speech_encoder_attention(speech)
        speech = speech.permute(1, 2, 0)  # Restauramos el orden de las dimensiones

        x = torch.cat((style, speech), dim=1)
        x = x.permute(2, 0, 1)  # Reordenamos las dimensiones para que sea compatible con la capa de atención
        x = self.decoder_attention(x)
        x = x.permute(1, 2, 0)  # Restauramos el orden de las dimensiones

        x = self.decoder(x)
        
        return x.squeeze(0) #, transcriptions
    
class WaveNetDiscriminator(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, sample_rate, dropout=0.2):
        super(WaveNetDiscriminator, self).__init__()
        self.wavenet = WaveNet(in_channels, in_channels, kernel_size, dilation, dropout)
        self.linear1 = nn.Linear(in_channels * sample_rate, out_channels)
        self.linear2 = nn.Linear(out_channels, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        output = self.wavenet(x)
        output = output.view(x.shape[0], -1)
        output = self.linear1(output)
        output = self.linear2(output)
        output = self.sigmoid(output)
        return output

class StyleAutoencoderModule(pl.LightningModule):

    def __init__(self, in_channels, out_channels, kernel_size, dilation, device, sample_rate, dropout=0.2):
        super(StyleAutoencoderModule, self).__init__()
        self.encoder_generator = StyleAutoEncoder(in_channels, out_channels, kernel_size, dilation, dropout)
        self.encoder_discriminator = WaveNetDiscriminator(in_channels, out_channels, kernel_size, dilation, sample_rate, dropout)
        self.decoder_generator = StyleAutoEncoder(in_channels, out_channels, kernel_size, dilation, dropout)
        self.decoder_discriminator = WaveNetDiscriminator(in_channels, out_channels, kernel_size, dilation, sample_rate, dropout)
        self.loss_speech_loss_fn = nn.MSELoss()
        self.loss_style_loss_fn = nn.MSELoss()
        self.disc_loss_fn = nn.BCEWithLogitsLoss()
        self.todevice = device

    def forward(self, speech, style):
        y = self.encoder_generator(speech, style)
        y1 = self.decoder_generator(y, style)
        enc_disc_out = self.encoder_discriminator(y)
        dec_disc_out = self.decoder_discriminator(y1)
        labels = torch.zeros(y.shape[0], 1, device=self.device)
        return y, y1, enc_disc_out, dec_disc_out, labels

    def training_step(self, batch, batch_idx):
        speech, style = batch
        if speech.ndim == 2: speech = speech.unsqueeze(1)
        if style.ndim == 2: style = style.unsqueeze(1)
        y, y1, enc_disc_out, dec_disc_out, labels = self(speech, style)
        loss_speech = self.loss_speech_loss_fn(y, speech)
        loss_style = self.loss_style_loss_fn(y1, style)
        loss_disc_speech = self.disc_loss_fn(enc_disc_out,labels) + self.disc_loss_fn(dec_disc_out,labels)
        loss = loss_speech + loss_style + loss_disc_speech
        self.log('train_loss', loss.item())
        return loss 
    
    def validation_step(self, batch, batch_idx):
        speech, style = batch
        if speech.ndim == 2: speech = speech.unsqueeze(1)
        if style.ndim == 2: style = style.unsqueeze(1)
        y, y1, enc_disc_out, dec_disc_out, labels = self(speech, style)
        loss_speech = self.loss_speech_loss_fn(y, speech)
        loss_style = self.loss_style_loss_fn(y1, style)
        loss_disc_speech = self.disc_loss_fn(enc_disc_out,labels) + self.disc_loss_fn(dec_disc_out,labels)
        loss = loss_speech + loss_style + loss_disc_speech
        self.log('val_loss', loss.item())
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-2)
        # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_loss'
        }

In [3]:
torch.set_float32_matmul_precision('medium') #  | 'high'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
batch_size = 4
sample_rate = 16000
metadata = pd.read_csv('metadata.csv')
dataset = AudioDataset(metadata, device, sample_rate=sample_rate)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [int(len(dataset) * 0.7), len(dataset) - int(len(dataset) * 0.7)])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, drop_last=True)
model = StyleAutoencoderModule(1, 64, 3, 12, device=device, sample_rate=sample_rate)
model = model.to(device)

In [5]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath='./checkpoints',
    filename='{epoch}-{val_loss:.2f}-{val_r2:.2f}',
    save_top_k=1,
    monitor='val_loss',
    every_n_epochs =1,
    mode='min'
)

trainer = pl.Trainer(
    accelerator='gpu',
    max_epochs=10,
    benchmark=True,
    # deterministic=True,
    precision=16,
    accumulate_grad_batches=6,
    callbacks=[
        checkpoint_callback,
        pl.callbacks.LearningRateMonitor(logging_interval='step')
    ]
)

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(model, train_loader, val_loader)

Missing logger folder: c:\Users\david\OneDrive\Documents\GitHub\Style-Transfer-VC-Model\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                  | Type                 | Params
---------------------------------------------------------------
0 | encoder_generator     | StyleAutoEncoder     | 1.9 M 
1 | encoder_discriminator | WaveNetDiscriminator | 1.0 M 
2 | decoder_generator     | StyleAutoEncoder     | 1.9 M 
3 | decoder_discriminator | WaveNetDiscriminator | 1.0 M 
4 | loss_speech_loss_fn   | MSELoss              | 0     
5 | loss_style_loss_fn    | MSELoss              | 0     
6 | disc_loss_fn          | BCEWithLogitsLoss    | 0     
---------------------------------------------------------------
5.9 M     Trainable params
0         Non-trainable params
5.9 M     Total params
11.894    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\trainer\call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
random_speech = train_dataset[0]
random_audio = random_speech[0]
style_audio = random_speech[1]

model.eval()
model = model.to(device)
random_audio = random_audio.unsqueeze(0)
style_audio = style_audio.unsqueeze(0)

with torch.no_grad():
    predicted_audio = model.encoder_generator(random_audio.unsqueeze(0), style_audio.unsqueeze(0))

print(random_audio.shape, style_audio.shape, predicted_audio.shape)

display(ipd.Audio(random_audio.cpu().numpy(), rate=sample_rate))
display(ipd.Audio(style_audio.cpu().numpy(), rate=sample_rate))
predicted_audio = predicted_audio.squeeze(0)
display(ipd.Audio(predicted_audio.cpu().numpy(), rate=sample_rate))

# print(random_audio)

torch.Size([1, 16000]) torch.Size([1, 16000]) torch.Size([1, 16000])


e:\miniconda3\envs\AI\lib\site-packages\IPython\lib\display.py:187: RuntimeWarning: invalid value encountered in divide
  scaled = data / normalization_factor * 32767
e:\miniconda3\envs\AI\lib\site-packages\IPython\lib\display.py:188: RuntimeWarning: invalid value encountered in cast
  return scaled.astype("<h").tobytes(), nchan


In [ ]:
plt.plot(random_audio.cpu().numpy())
plt.plot(style_audio.cpu().numpy())
plt.plot(predicted_audio.cpu().numpy())
plt.show()